In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option("display.max_column",100)
pd.set_option("expand_frame_repr",False)

In [3]:
off_train_file=r"D:\dataset\o2o\ccf_offline_stage1_train.csv"
off_test_file=r"D:\dataset\o2o\ccf_offline_stage1_test_revised.csv"
on_train_file=r"D:\dataset\o2o\ccf_online_stage1_train.csv"
off_train=pd.read_csv(off_train_file)
off_test=pd.read_csv(off_test_file)
on_train=pd.read_csv(on_train_file)

In [ ]:
#工具函数
def add_agg_feature_name(df,df_group,group_cols,value_col,agg_ops,col_names):
    df_group[value_col]=df_group[value_col].astype("float")
    df_agg=pd.DataFrame(df_group.groupby(group_cols)[value_col].agg(agg_ops)).reset_index()
    df_agg.columns=group_cols+col_names
    df=df.merge(df_agg,on=group_cols,how="left")
    return df;